In [ ]:
# Dependencies
from splinter import Browser
import requests
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import os
import pandas as pd
from splinter import Browser
import time

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### latest news title and description

In [ ]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [ ]:
html =browser.html
news_soup = BeautifulSoup(html, 'html.parser')


In [ ]:
print(news_soup.prettify())

In [ ]:
# pull latest news title and paragrapgh
slide_elem = news_soup.select_one('ul.item_list li.slide')
title  = slide_elem.find('div', class_='content_title')

paragraph  = slide_elem.find('div', class_='article_teaser_body')

print('-----------------------------------------------------------------------------')
print('News_title = '+title.text.strip())
print('')
print('News_p = '+paragraph.text.strip())
print('-----------------------------------------------------------------------------')

    




### JPL Mars Space Images - Featured Image

In [ ]:
# images
image_url = 'https://www.jpl.nasa.gov/spaceimages/'
browser.visit(image_url) 


In [ ]:
html =browser.html
image_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
print(image_soup.prettify())

In [ ]:
# Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [ ]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [ ]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

img_url = image_soup.select_one("figure.lede a img").get("src")

print(f"featured_image_url = https://www.jpl.nasa.gov{img_url}")

### Mars Weather

In [ ]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'


In [ ]:
twitter_response = requests.get(twitter_url)
twitter_soup = BeautifulSoup(twitter_response.text, 'lxml')

In [ ]:
print(twitter_soup.prettify())

In [ ]:
twitter_result = twitter_soup.find('div', class_='js-tweet-text-container')
print(f"mars_weather ={twitter_result.text.strip()}")

### Mars Facts

In [ ]:
#   scrape the table containing facts about the planet including Diameter, Mass, etc
space_data = pd.read_html('https://space-facts.com/mars/')[0]
space_data.columns=['Description','Value']
space_data.set_index("Description",inplace=True)
space_data.head()

In [ ]:
# Export pandas df to html script
marsfacts = space_data.to_html()
marsfacts.replace("\n", "")
space_data.to_html('marsfacts.html')

### Mars Hemispheres

In [ ]:
# Scrape astrogeology.usgs.gov for hemisphere image urls and titles
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html, 'lxml')
base_url ="https://astrogeology.usgs.gov"

image_list = hemisphere_soup.find_all('div', class_='item')

# Create list to store dictionaries of data
hemisphere_image_urls = []

# Loop through each hemisphere and click on link to find large resolution image url
for image in image_list:
    hemisphere_dict = {}
    
    href = image.find('a', class_='itemLink product-item')
    link = base_url + href['href']
    browser.visit(link)
    
    time.sleep(1)
    
    hemisphere_html2 = browser.html
    hemisphere_soup2 = BeautifulSoup(hemisphere_html2, 'lxml')
    
    img_title = hemisphere_soup2.find('div', class_='content').find('h2', class_='title').text
    hemisphere_dict['title'] = img_title
    
    img_url = hemisphere_soup2.find('div', class_='downloads').find('a')['href']
    hemisphere_dict['url_img'] = img_url
    
    # Append dictionary to list
    hemisphere_image_urls.append(hemisphere_dict)
      
hemisphere_image_urls